# Init

In [1]:
import os, sys

ROOT_DIR = '../..'
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

from dotenv import load_dotenv
load_dotenv('.env')

DATAPATH = os.getenv("DATAPATH")

import pandas as pd 

In [2]:
filenames = [f for f in os.listdir(DATAPATH) if f.endswith('.csv')]
filenames

['amostra_projeto1.csv',
 'amostra_projeto_3.csv',
 'projeto_4.csv',
 'amostra_projeto_2.csv']

# Carga e otimização

In [3]:
for filename in filenames:
    filepath = os.path.join(DATAPATH, filename)
    !wc -l $filepath
    !head -n 20 $filepath
    print()
    !tail $filepath
    print('\n')

163 src/datasets/amostra_projeto1.csv
DADOS,,,,,,,,,,,,,
Titulo,Projeto 1 - Wa,,,,,,,,,,,,
Tipologia,Térrea,,,,,,,,,,,,
Área Terreno,"485,00",,,,,,,,,,,,
Área Construída,"246,63",,,,,,,,,,,,
Área Fundação,"237,98",,,,,,,,,,,,
Área Fachada,"597,88",,,,,,,,,,,,
Área Parede,"615,86",,,,,,,,,,,,
Qtde BWCs,"5,00",,,,,,,,,,,,
,,,,,,,,,,,,,
TABELA DE ORÇAMENTO,,,,,,,,,,,,,
Item,Referência,Tipo,Código,Descrição,Unid.,Quantidade,BDI,Preço Material,,Preço Execução,,Preço,
,,,,,,,,Unitário,Total,Unitário,Total,Unitário,Total
1.,,,,ETAPAS PRE OBRA,,,"0,0%",,,," R$  8.086,25 ",," R$  8.086,25 "
1.1.,,,,SONDAGEM,,,"0,0%",,,," R$  3.000,00 ",," R$  3.000,00 "
1.1.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-SON,SONDAGEM,VB,1,"0,0%",,," R$  3.000,00 "," R$  3.000,00 "," R$  3.000,00 "," R$  3.000,00 "
1.2.,,,,LEVANTAMENTO TOPOGRAFICO,,,"0,0%",,,," R$  1.200,00 ",," R$  1.200,00 "
1.2.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-LVT,LEVANTAMENTO TOPOGRAFICO,VB,1,"0,0%",,," R$  1.200,00 "," R$  1.200,00 "," R$  1.200,0

## Features

In [4]:
def transpose_dataframe(df:pd.DataFrame):
    df = df.set_index('variable').T
    df = df.reset_index(drop=True)
    df.columns.name = None
    return df

In [5]:
df_features = pd.DataFrame(data=None)
for filename in filenames:
    df = pd.read_csv(os.path.join(DATAPATH, filename), skiprows=1, nrows=8, usecols=[0,1], header=None, names=['variable', 'value'])
    df = transpose_dataframe(df)
    df['filename'] = filename
    df_features = pd.concat([df_features, df])
df_features = df_features.sort_values(by='Titulo').reset_index(drop=True).reset_index(names=['id_project'])
df_features

,id_project,Titulo,Tipologia,Área Terreno,Área Construída,Área Fundação,Área Fachada,Área Parede,Qtde BWCs,filename
0,0,Projeto 1 - Wa,Térrea,"485,00","246,63","237,98","597,88","615,86","5,00",amostra_projeto1.csv
1,1,Projeto 2 - Ca,Sobrado,"300,00","249,24","161,55","747,17","662,06","4,00",amostra_projeto_2.csv
2,2,Projeto 3 - Je2,Sobrado,"500,00","425,00","378,31","521,67","581,76","6,00",amostra_projeto_3.csv
3,3,Projeto 4 - Je3,Sobrado,"250,00","257,00","221,36","430,11","547,27","6,00",projeto_4.csv


In [6]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id_project       4 non-null      int64 
 1   Titulo           4 non-null      object
 2   Tipologia        4 non-null      object
 3   Área Terreno     4 non-null      object
 4   Área Construída  4 non-null      object
 5   Área Fundação    4 non-null      object
 6   Área Fachada     4 non-null      object
 7   Área Parede      4 non-null      object
 8   Qtde BWCs        4 non-null      object
 9   filename         4 non-null      object
dtypes: int64(1), object(9)
memory usage: 452.0+ bytes


In [7]:
numeric_columns = df_features.columns[3:-1]
for col in numeric_columns:
    df_features[col] = df_features[col].str.replace('.', '').str.replace(',', '.')
    df_features[col] = pd.to_numeric(df_features[col])
df_features

,id_project,Titulo,Tipologia,Área Terreno,Área Construída,Área Fundação,Área Fachada,Área Parede,Qtde BWCs,filename
0,0,Projeto 1 - Wa,Térrea,485.0,246.63,237.98,597.88,615.86,5.0,amostra_projeto1.csv
1,1,Projeto 2 - Ca,Sobrado,300.0,249.24,161.55,747.17,662.06,4.0,amostra_projeto_2.csv
2,2,Projeto 3 - Je2,Sobrado,500.0,425.00,378.31,521.67,581.76,6.0,amostra_projeto_3.csv
3,3,Projeto 4 - Je3,Sobrado,250.0,257.00,221.36,430.11,547.27,6.0,projeto_4.csv


In [8]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_project       4 non-null      int64  
 1   Titulo           4 non-null      object 
 2   Tipologia        4 non-null      object 
 3   Área Terreno     4 non-null      float64
 4   Área Construída  4 non-null      float64
 5   Área Fundação    4 non-null      float64
 6   Área Fachada     4 non-null      float64
 7   Área Parede      4 non-null      float64
 8   Qtde BWCs        4 non-null      float64
 9   filename         4 non-null      object 
dtypes: float64(6), int64(1), object(3)
memory usage: 452.0+ bytes


In [9]:
from unicodedata import normalize


def transliterate(string):
  words = normalize('NFKD', string).encode('ASCII','ignore').decode('ASCII').split()
  return '_'.join(words)

def parse_columns(columns):
  columns = transliterate(', '.join(columns)).lower().split(',')
  return [col.strip('_') for col in columns]

In [10]:
df_features.columns = parse_columns(df_features.columns)
df_features

,id_project,titulo,tipologia,area_terreno,area_construida,area_fundacao,area_fachada,area_parede,qtde_bwcs,filename
0,0,Projeto 1 - Wa,Térrea,485.0,246.63,237.98,597.88,615.86,5.0,amostra_projeto1.csv
1,1,Projeto 2 - Ca,Sobrado,300.0,249.24,161.55,747.17,662.06,4.0,amostra_projeto_2.csv
2,2,Projeto 3 - Je2,Sobrado,500.0,425.00,378.31,521.67,581.76,6.0,amostra_projeto_3.csv
3,3,Projeto 4 - Je3,Sobrado,250.0,257.00,221.36,430.11,547.27,6.0,projeto_4.csv


In [11]:
df_features.to_parquet(os.path.join(DATAPATH, 'staged', 'features.parquet'))

## Target

In [12]:
def merge_columns(df:pd.DataFrame):
    columns_to_rename = {
        "Preço Material": 'Preço Material Unitário',
        'Unnamed: 9': 'Preço Material Total',
        'Preço Execução': 'Preço Execução Unitário',
        'Unnamed: 11': 'Preço Execução Total',
        'Preço': 'Preço Unitário',
        'Unnamed: 13': 'Preço Total',
    }
    df = df.rename(columns=columns_to_rename)
    df = df.drop(0).reset_index(drop=True)
    return df

In [13]:
df_target = pd.DataFrame(data=None)
for filename in filenames:
    if filename == 'projeto_4.csv':
        continue
    df = pd.read_csv(os.path.join(DATAPATH, filename), skiprows=11)
    df = merge_columns(df)
    df['id_project'] = df_features.query('filename == @filename')['id_project'].item()
    df_target = pd.concat([df_target, df])
df_target.columns = parse_columns(df_target.columns)
df_target

,item,referencia,tipo,codigo,descricao,unid.,quantidade,bdi,preco_material_unitario,preco_material_total,preco_execucao_unitario,preco_execucao_total,preco_unitario,preco_total,id_project
0,1.,NaN,NaN,NaN,ETAPAS PRE OBRA,NaN,NaN,"0,0%",NaN,NaN,NaN,"R$ 8.086,25",NaN,"R$ 8.086,25",0
1,1.1.,NaN,NaN,NaN,SONDAGEM,NaN,NaN,"0,0%",NaN,NaN,NaN,"R$ 3.000,00",NaN,"R$ 3.000,00",0
2,1.1.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-SON,SONDAGEM,VB,1,"0,0%",NaN,NaN,"R$ 3.000,00","R$ 3.000,00","R$ 3.000,00","R$ 3.000,00",0
3,1.2.,NaN,NaN,NaN,LEVANTAMENTO TOPOGRAFICO,NaN,NaN,"0,0%",NaN,NaN,NaN,"R$ 1.200,00",NaN,"R$ 1.200,00",0
4,1.2.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-LVT,LEVANTAMENTO TOPOGRAFICO,VB,1,"0,0%",NaN,NaN,"R$ 1.200,00","R$ 1.200,00","R$ 1.200,00","R$ 1.200,00",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,16.,NaN,NaN,NaN,FORMALIZACAO DA ENTREGA,NaN,NaN,"0,0%",NaN,NaN,NaN,"R$ 912,85",NaN,"R$ 912,85",1
157,16.1.,NaN,NaN,NaN,HABITE-SE E AVERBACAO,NaN,NaN,"0,0%",NaN,NaN,NaN,"R$ 912,85",NaN,"R$ 912,85",1
158,16.1.1,MKS_COMPOSICOES,COMPOSICAO,CMP_EPO-TAX-AVR,TAXA AVERBACAO,VB,1,"0,0%",NaN,NaN,"R$ 502,85","R$ 502,85","R$ 502,85","R$ 502,85",1
159,16.1.2,MKS_COMPOSICOES,COMPOSICAO,CMP_EPO-TAX-HBT,TAXA HABITE-SE PREFEITURA,VB,1,"0,0%",NaN,NaN,"R$ 150,00","R$ 150,00","R$ 150,00","R$ 150,00",1


In [14]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
Index: 468 entries, 0 to 160
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   item                     468 non-null    object
 1   referencia               280 non-null    object
 2   tipo                     280 non-null    object
 3   codigo                   280 non-null    object
 4   descricao                468 non-null    object
 5   unid.                    280 non-null    object
 6   quantidade               280 non-null    object
 7   bdi                      468 non-null    object
 8   preco_material_unitario  220 non-null    object
 9   preco_material_total     367 non-null    object
 10  preco_execucao_unitario  208 non-null    object
 11  preco_execucao_total     377 non-null    object
 12  preco_unitario           280 non-null    object
 13  preco_total              468 non-null    object
 14  id_project               468 non-null    int64 

In [15]:
def parse_price_columns(df:pd.DataFrame, price_columns):
    for col in price_columns:
        df[col] = df[col].str.strip().str.split(r'\s').str[-1]
        df[col] = df[col].str.replace('.', '').str.replace(',', '.')
        df[col] = pd.to_numeric(df[col])
    return df

price_columns = [col for col in df_target.columns if 'preco' in col]
df_target = parse_price_columns(df_target, price_columns)
df_target['quantidade'] = pd.to_numeric(df_target['quantidade'].str.replace(',', '.'))
df_target['bdi'] = df_target['bdi'].str.replace('%', '').str.replace(',', '.')
df_target['bdi'] = pd.to_numeric(df_target['bdi'])
df_target.reset_index(drop=True, inplace=True)
df_target

,item,referencia,tipo,codigo,descricao,unid.,quantidade,bdi,preco_material_unitario,preco_material_total,preco_execucao_unitario,preco_execucao_total,preco_unitario,preco_total,id_project
0,1.,NaN,NaN,NaN,ETAPAS PRE OBRA,NaN,NaN,0.0,NaN,NaN,NaN,8086.25,NaN,8086.25,0
1,1.1.,NaN,NaN,NaN,SONDAGEM,NaN,NaN,0.0,NaN,NaN,NaN,3000.00,NaN,3000.00,0
2,1.1.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-SON,SONDAGEM,VB,1.0,0.0,NaN,NaN,3000.00,3000.00,3000.00,3000.00,0
3,1.2.,NaN,NaN,NaN,LEVANTAMENTO TOPOGRAFICO,NaN,NaN,0.0,NaN,NaN,NaN,1200.00,NaN,1200.00,0
4,1.2.1,MKS_COMPOSICOES,COMPOSICAO,CMP_INF-LVT,LEVANTAMENTO TOPOGRAFICO,VB,1.0,0.0,NaN,NaN,1200.00,1200.00,1200.00,1200.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,16.,NaN,NaN,NaN,FORMALIZACAO DA ENTREGA,NaN,NaN,0.0,NaN,NaN,NaN,912.85,NaN,912.85,1
464,16.1.,NaN,NaN,NaN,HABITE-SE E AVERBACAO,NaN,NaN,0.0,NaN,NaN,NaN,912.85,NaN,912.85,1
465,16.1.1,MKS_COMPOSICOES,COMPOSICAO,CMP_EPO-TAX-AVR,TAXA AVERBACAO,VB,1.0,0.0,NaN,NaN,502.85,502.85,502.85,502.85,1
466,16.1.2,MKS_COMPOSICOES,COMPOSICAO,CMP_EPO-TAX-HBT,TAXA HABITE-SE PREFEITURA,VB,1.0,0.0,NaN,NaN,150.00,150.00,150.00,150.00,1


In [17]:
df_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 468 entries, 0 to 467
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   item                     468 non-null    object 
 1   referencia               280 non-null    object 
 2   tipo                     280 non-null    object 
 3   codigo                   280 non-null    object 
 4   descricao                468 non-null    object 
 5   unid.                    280 non-null    object 
 6   quantidade               280 non-null    float64
 7   bdi                      468 non-null    float64
 8   preco_material_unitario  220 non-null    float64
 9   preco_material_total     367 non-null    float64
 10  preco_execucao_unitario  208 non-null    float64
 11  preco_execucao_total     377 non-null    float64
 12  preco_unitario           280 non-null    float64
 13  preco_total              468 non-null    float64
 14  id_project               4

In [16]:
df_target.to_parquet(os.path.join(DATAPATH, 'staged', 'target.parquet'))